# w261 Final Project - Clickthrough Rate Prediction


- Team 15
- Erik Hou, Noah Pflaum, Connor Stern, Anu Yadav
- Fall 2019, section 3  

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

In [2]:
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import math

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [5]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "fp_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [6]:
spark

# __Section 1__ - Question Formulation

The digital advertising industry is booming. Every day, advertisers publish billions of ads to websites and other online platforms where potential customers can view and click on them. For example, Nike as an advertiser might post an ad for running shoes on the ESPN.com website or in a Google search for shoes. The position of the ad, the audience, and the time that the ad is displayed may all significantly influence a user's decision to click on it or not. 

Ad posts generate vast amount of user log data representing ad placement and whether or not a user clicked on an ad. This data is a treasure trove for companies seeking to more effectively target their ads to specific consumers. As such 'clickthrough rate' prediction (determining whether or not a customer will click on an ad) has become a core task in the field of digital advertising, and multiple clickthrough rate prediction challenges have been launched in recent years to build and continuously improve models that can predict given the features of the ad, whether or not a user will click on the ad. 

In this project, we seek to build our own homegrown model to complete such a task, inspired by previous successful models. A baseline for any model would be to measure the average click-through rate (CTR, percent of ads clicked).

We therefore begin with the question:

-  What are some common approaches to CTR prediction? Which methods allow for more complex interactions among features? Which have been the most successful in previous competitions?
    -  After some initial research, we find that CTR prediction is typically treated as a logistic regression problem. However, there are many different methods *within* a logistic regression framework that the problem can be handled. One such method is that of a Field-Aware Factorization Machine (FFM), which was the winning model for several of the previously mentioned CTR prediction competitions.

Inspired by the success of Juan, Zhuang, Chin, & Lin (the authors of several papers on FFMs and winners of the CTR challenges), we decide to implement a FFM model of our own. This leads us to several additional questions we seek to answer:

-  What resource requirements are necessary for implementing an FFM?

-  Can the FFM model be implemented in Pyspark using distributed computing using map-reduce patterns? What level of performance can we expect with this implementation?

    -  Juan, Zhuang, Chin, & Lin have built their own custom library called 'LibFFM'. Their papers outline an approach using the Hogwild algorithm that allows for concurrent updates to a sparse matrix of model coefficients, which eliminates some of the issues with the communication and synchronization barriers that generally exist when using stochastic gradient descent. Our goal is to explore how this algorithm might be implemented using the techniques we learned in this class (such as Pyspark and distributed computing using map-reduce patterns, as mentioned above). We do not expect our algorithm to be as performant as a customized C++ implmentation; instead, we highlight the application of the ideas learned in this class for creating predicitve models for big data.

# __Section 2__ - Algorithm Explanation

Because an online advertisement can either be clicked ($response = 1$) or not ($response = 0$) , Click-Through-Rate (CTR) Prediction is generally treated as a logistic regression problem. For any set of features, we calculate some value $s$, and perform the logit transformation to yield our CTR prediction:
$$CTR = \frac{1}{1+e^{-s}} $$

There are several methods we can use to estimate $s$, each with its own benefits and drawbacks. Typical implementations include a linear model, a degree-2 polynomial mapping, a factorization machine, and a field-aware factorization machine. 

We will consider an example with the following dataset as we discuss the different methods of estimating $s$:

<table>
<th>Response</th>
<th>Publisher</th>
<th>Advertiser</th>
<th>Gender</th>
<tr><td>1</td><td>Netflix</td><td>Pepsi</td><td>Male</td></tr>
<tr><td>0</td><td>Spotify</td><td>Pepsi</td><td>Male</td></tr>
<tr><td>0</td><td>Facebook</td><td>Gatorade</td><td>Female</td></tr>
<tr><td>1</td><td>Spotify</td><td>Coca-cola</td><td>Male</td></tr>
<tr><td>1</td><td>Facebook</td><td>Coca-cola</td><td>Female</td></tr>
<tr><td>0</td><td>Facebook</td><td>Pepsi</td><td>Female</td></tr>
<tr><td>1</td><td>Netflix</td><td>Gatorade</td><td>Female</td></tr>
</table>

In this dataset, we refer to the categories Publisher, Advertiser, and Gender as "fields" and the labels within each field (Netflix, Spotify, Facebook, Pepsi, Gatorade, Coca-cola, Male, Female) as "features."

#### Linear Model
In a linear model, the algorithm learns a weight for every given feature. The formulation of the model is:
$$s = \phi(\textbf{w},\textbf{x}) =\textbf{w}^T \textbf{x}  =\sum_{j \epsilon C_1}w_jx_j$$
where $\textbf{w}$ is the learned model, $\textbf{x}$ is the data observation, and $C_1$ is the non-zero elements in $\textbf{x}$. 

In our toy example, our model would learn different weights for the different Publishers (Netflix, Spotify, and Pepsi), Advertisers (Pepsi, Gatorade, and Coca-cola), and Genders (Male, Female). The value $s$ would then be calculated for each impression using these different weights. Thus, for each impression we would have have:

$$
\begin{aligned}
s =& w_{Netflix}\cdot x_{Netflix} + w_{Spotify}\cdot x_{Spotify} + w_{Facebook}\cdot x_{Facebook} \\
&+ w_{Coca-cola}\cdot x_{Coca-cola} + w_{Pepsi}\cdot x_{Pepsi} + w_{Gatorade}\cdot x_{Gatorade} \\
&+ + w_{Male}\cdot x_{Male} + w_{Female}\cdot x_{Female}
\end{aligned}
$$

For our first impression of the dataset (Netflix, Pepsi, Male) this becomes:
$$s = w_{Netflix} + w_{Coca-cola} + w_{Male}$$
since $x_j = 1$ for Netflix, Pepsi, and Male while $x_j = 0$ for all other features. 


This model is simple and efficient, yet it does not allow for interactive effects between features. For example, Coca-cola may have a higher CTR with Netflix than another publisher. A linear model is unable to learn this type of information, as it essentially learns the "average effect" of each feature.

#### Degree-2 Polynomial Mapping
The simplest way to learn the effect of the "feature conjunction" described above (in the case where a particular advertiser may have a higher CTR with one publisher compared to others) is to use a degree-2 polynomial mapping. In this model, the algorithm learns an additional weight for each feature pair. The formulation of the model is:
$$s = \phi(\textbf{w},\textbf{x}) = \sum_{j_1, j_2 \epsilon C_2} w_{j_1,j_2} \cdot x_{j_1}x_{j_2}$$
where $C_2$ is the pairwise combination of non-zero elements in $\textbf{x}$.


Returning to our example dataset and the impression with the features Netflix, Pepsi, and Male the model would be:
$$s = w_{Netflix} + w_{Pepsi} + w_{Male} + w_{Netflix,Pepsi} + w_{Netflix,Male} + w_{Pepsi,Male}$$

While this model improves on the linear model by allowing us to account for interactions between features, it does not handle sparse datasets well. Since we have 0 impressions of the advertiser Gatorade with the publisher Spotify, the model prediction will be trivial as no weight was learned for this feature combination. The model is also susceptible to overfitting, as it generates unreliable predictions for feature combinations with a very small number of impressions.

#### Factorization Machine
Factorization Machines (FM) begin to provide us a solution to the problem of sparse datasets. Here the model learns a latent vector, rather than an explicit weight, for each feature. The model formulation is:
$$s = \phi(\textbf{w},\textbf{x}) = \sum_{j_1, j_2 \epsilon C_2} \langle \textbf{w}_{j_1}, \textbf{w}_{j_2} \rangle x_{j_1}x_{j_2}$$
where $\textbf{w}_{j_1}$ and $\textbf{w}_{j_2}$ are two learned latent vectors of length $k$ (some user-defined parameter).

Returning to our example of an impression with the features of Netflix, Pepsi, and Male, the FM model would be:
$$s = w_{Netflix} \cdot w_{Pepsi} + w_{Netflix} \cdot w_{Male} + w_{Pepsi} \cdot w_{Male}$$
where $w_{Netflix}, w_{Pepsi}, w_{Male} \epsilon {\rm I\!R}^k$.

This allows the model to learn the latent vectors for each feature based on all the data points for that feature, and these latent vectors can be used to predict the CTR for unobserved feature combinations (such as Spotify and Gatorade, as previously mentioned), something the degree-2 polynomial mapping method did not allow us to do.

We notice in our example above, however, that the latent vector $w_{Netflix}$ is used twice: once to calculate the latent effect of the Publisher Netflix with the Advertiser Pepsi ($w_{Netflix} \cdot w_{Pepsi}$), and once to calculate the latent effect of the Publisher Netflix with the Gender Male ($w_{Netflix} \cdot w_{Male}$). Yet the latent effect for publisher with advertiser could be very different from the latent effect for publisher with gender, and as such the Factorization Machine model is too restrictive and unrealistic.


#### Field-Aware Factorization Machine
Field-Aware Factorization Machines (FFM) provide a solution to this problem by introducing the flexibility to learn multiple latent vectors for each feature. In the context of the above example, the FFM model will learn two separate latent vectors for Netflix: $w_{Netflix, A}$, which will be used for the calculation of $P \times A$ (to learn the latent effect of Netflix with a given advertiser), and $w_{Netflix, G}$. $w_{Netflix, A}$, for the calculation of $P \times G$ (to learn the latent effect of Netflix with a given gender).

Specifically, the FFM model formulation with the (Netflix, Pepsi, Male) impression gives us:

$$s = w_{Netflix, A} \cdot w_{Pepsi, P} + w_{Netflix, G} \cdot w_{Male, P} + w_{Pepsi, G} \cdot w_{Male, A}$$

Now, the latent effect of (Netflix, Pepsi), is learned by using the latent vector $w_{Netflix, A}$, since Pepsi belongs to the advertiser field (A). The latent effect of (Netflix, Male), by contrast, is learned using a different latent vector, $w_{Netflix, G}$, since Male belongs to the gender field (G). In this way, FFM splits the latent factors for $P \times A$ and $P \times G$, something the traditional Factorization Machine is unable to do. 

The full model formulation for a Field-Aware Factorization Machine is:
$$s = \phi(\textbf{w},\textbf{x}) = \sum_{j_1, j_2 \epsilon C_2} \langle \textbf{w}_{j_1, f_2}, \textbf{w}_{j_2, f_1} \rangle x_{j_1}x_{j_2}$$

where $f_1$ and $f_2$ are the fields of $j_1$ and $j_2$, respectively. 
#### Optimization

As part of a logistic regression problem, the goal is to find the set of parameters that minimize the log-loss function, defined by:

$$ LogLoss = - \frac{1}{n} \sum_{i=1}^n [y_i \cdot log_e(\hat{y_i}) + (1-y_i) \cdot log_e(1-\hat{y_i})] $$


where $n$ is the number of impressions, $y_i$ is the true CTR of impression $i$, and $\hat{y_i}$ is the predicted CTR of impression $i$, $\frac{1}{1+e^{-s_i}}$.

For a given impression $x_i$, we have two cases:
* If $y-i = 1$:
$$
\begin{aligned}
loss &= -log_e(\hat{y_i}) \\
&= -log_e \left( \frac{1}{1+e^{-s}} \right) \\
&= log_e (1+e^{-s})
\end{aligned}
$$

* If $y_i = 0$:
$$
\begin{aligned}
loss &= -log_e(1-\hat{y_i}) \\
&= -log_e \left( 1 - \frac{1}{1+e^{-s}} \right)\\
&= -log_e \left( \frac{1 + e^{-s} - 1}{1+e^{-s}} \right) \\
&= -log_e \left( \frac{e^-s}{1+e^{-s}} \right) \\
&= -log_e \left( \frac{1}{1+e^{s}} \right) \\
&= log_e(1+e^{s})
\end{aligned}
$$

As such, the loss for impression $x_i$ can be written as:
$$log(1+exp(-\bar{y_i}\cdot s_i)$$
where 
$$\bar{y_i} = 
\begin{cases}
    1,\ if\ y_i=1\\
    -1,\ if\ y_i=0\\
\end{cases}$$

Therefore, we can rewrite the log-loss function as:
$$ Loss = \frac{1}{n} \sum_{i=1}^n log(1+exp(-\bar{y_i}\cdot s_i))$$
where 
$\bar{y_i} = 
\begin{cases}
    1,\ if\ y_i=1\\
    -1,\ if\ y_i=0\\
\end{cases}$


and $s = \phi(\textbf{w},\textbf{x}_i) = \sum_{j_1, j_2 \epsilon C_2} \langle \textbf{w}_{j_1, f_2}, \textbf{w}_{j_2, f_1} \rangle x_{j_1}x_{j_2}$


Introducing regularization, the optimization problem we have is:

$$ \min_{\textbf{w}} \sum_{i=1}^m \left( log(1+exp(-\bar{y_i}\phi(\textbf{w},\textbf{x}_i)) + \frac{\lambda}{2}\|\textbf{w}\|^2 \right)$$

$m$ is the number of impressions, and  $\lambda$ is our regularization parameter.

We solve this optimization problem using gradient descent methods.


In [ ]:
def get_CTR(s):
    return np.divide(1,1+exp(-s))

#### Toy example: FFM in action

We will now walk through an entire iteration of the FFM algorithm to learn a CTR prediction model.
For our work here, we will utilize a hashed representation of our dataset. In this representation, each field is represented by the first integer in the hash, and the feature within that field is represented by the second integer. The Publisher field is represented by the integer 1, the Advertiser field by the integer 2, and the Gender field by the integer 3. Within the Publisher field, Netflix is represented by the integer 1, Spotify by 2, and Facebook by 3. Within the Advertiser field, Pepsi is represented by the integer 1, Gatorade by 2, and Coca-cola by 3. Within the Gender field, Male is represented by 1 and Female by 2. Using this hash representation, our first impression of (Netflix, Pepsi, Male) has the features 1:1, 2:1, 3:1.
The full hashed representation of our dataset is shown below:

<table>
<th>Impression</th>
<th>Response</th>
<th>Features</th>
<tr><td>1</td><td>1</td><td>1:1, 2:1, 3:1</td></tr>
<tr><td>2</td><td>1</td><td>1:2, 2:1, 3:1</td></tr>
<tr><td>3</td><td>0</td><td>1:3, 2:2, 3:2</td></tr>
<tr><td>4</td><td>0</td><td>1:2, 2:3, 3:1</td></tr>
<tr><td>5</td><td>1</td><td>1:3, 2:3, 3:2</td></tr>
<tr><td>6</td><td>0</td><td>1:3, 2:1, 3:2</td></tr>
<tr><td>7</td><td>1</td><td>1:1, 2:2, 3:2</td></tr>
<table>

In [57]:
%%writefile data/toy_set.txt
1	1	1	1	1
2	1	2	1	1
3	0	3	2	2
4	0	2	3	1
5	1	3	3	2
6	0	3	1	2
7	1	1	2	2

Overwriting data/toy_set.txt


In [7]:
toyRDD = sc.textFile('data/toy_set.txt')

__1)__ We define k (the length of our latent vectors) to be 3.

__2)__ We randomly initialize our latent vectors $\textbf{w}$. For the Publisher field, each feature will have two different latent vectors (represented by columns below)--one corresponding to Advertiser (A), and one corresponding to Gender (G):

In [24]:
#W_P = np.random.uniform(-1.7,1.7, size=(6,3)).round(1)
#W_P
W_P =np.array([0.9, -0.2, -0.2, -0.6, 0, -0.3, -0.1, -0.5, -1.2, -1.1, 0.2, 1.2, 1.4,-0.7, 0.7, 0, 0.8, 0.1]).reshape(6,3)
W_P 

array([[ 0.9, -0.2, -0.2],
       [-0.6,  0. , -0.3],
       [-0.1, -0.5, -1.2],
       [-1.1,  0.2,  1.2],
       [ 1.4, -0.7,  0.7],
       [ 0. ,  0.8,  0.1]])

<table>
<th>$\textbf{w}_{Netflix,A}$</th>
<th>$\textbf{w}_{Netflix,G}$</th>
<th>$\textbf{w}_{Spotify,A}$</th>
<th>$\textbf{w}_{Spotify,G}$</th>
<th>$\textbf{w}_{Facebook,A}$</th>
<th>$\textbf{w}_{Facebook,G}$</th>
<tr><td>0.9</td><td>-0.6</td><td>-0.1</td><td>-1.1</td><td>1.4</td><td>0</td></tr>
<tr><td>-0.2</td><td>0</td><td>-0.5</td><td>0.2</td><td>-0.7</td><td>0.8</td></tr>
<tr><td>-0.2</td><td>-0.3</td><td>-1.2</td><td>1.2</td><td>0.7</td><td>0.1</td></tr>
<table>
The Advertiser field will similarly have two different latent vectors for each feature--one for Publisher (P), and one for Gender (G):

In [25]:
#W_A = np.random.uniform(-1.7,1.7, size=(6,3)).round(1)
#W_A
W_A =np.array([1, -0.3, -0.1, -0.4, -1, 0, -0.2, -1, -1.7, 0.7, -0.3, -1.1, 0.1, 1.3, -0.2, 0.5, 0.2, 0.5]).reshape(6,3)
W_A 

array([[ 1. , -0.3, -0.1],
       [-0.4, -1. ,  0. ],
       [-0.2, -1. , -1.7],
       [ 0.7, -0.3, -1.1],
       [ 0.1,  1.3, -0.2],
       [ 0.5,  0.2,  0.5]])

<table>
<th>$\textbf{w}_{Pepsi,P}$</th>
<th>$\textbf{w}_{Pepsi,G}$</th>
<th>$\textbf{w}_{Gatorade,P}$</th>
<th>$\textbf{w}_{Gatorade,G}$</th>
<th>$\textbf{w}_{Coca-cola,P}$</th>
<th>$\textbf{w}_{Coca-cola,G}$</th>
<tr><td>1</td><td>-0.4</td><td>-0.2</td><td>0.7</td><td>0.1</td><td>0.5</td></tr>
<tr><td>-0.3</td><td>-1</td><td>-1</td><td>-0.3</td><td>-1.3</td><td>0.2</td></tr>
<tr><td>-0.1</td><td>0</td><td>-1.7</td><td>-1.1</td><td>-0.2</td><td>0.5</td></tr>
<table>

The Gender field will also have two different latent vectors per feature (one for Publisher (P), one for Advertiser (A)): 

In [26]:
#W_G = np.random.uniform(-1.7,1.7, size=(4,3)).round(1)
#W_G
W_G =np.array([-0.4, 0.8, 1.5, -0.9, 1.3, -1, 0.8, -1.5, 0.3, 0.4, 0.7, 0.5]).reshape(4,3)
W_G

array([[-0.4,  0.8,  1.5],
       [-0.9,  1.3, -1. ],
       [ 0.8, -1.5,  0.3],
       [ 0.4,  0.7,  0.5]])

<table>
<th>$\textbf{w}_{Male,P}$</th>
<th>$\textbf{w}_{Male,A}$</th>
<th>$\textbf{w}_{Female,P}$</th>
<th>$\textbf{w}_{Female,A}$</th>
<tr><td>-0.4</td><td>-0.9</td><td>0.8</td><td>0.4</td></tr>
<tr><td>0.8</td><td>1.3</td><td>-1.5</td><td>0.7</td></tr>
<tr><td>1.5</td><td>-1</td><td>0.3</td><td>0.5</td></tr>
<table>

__3)__ We calculate $s$ for each impression using these weight vectors.

For each impression, we would use the formula
$$s = \phi(\textbf{w},\textbf{x}) = \sum_{j_1, j_2 \epsilon C_2} \langle \textbf{w}_{j_1, f_2}, \textbf{w}_{j_2, f_1} \rangle x_{j_1}x_{j_2}$$    
to generate our value of $s$. Fully expanded, this would be:
    
$$
\begin{aligned}
s =& \langle\textbf{w}_{Netflix,A} \cdot \textbf{w}_{Pepsi,P}\rangle x_{Netflix}x_{Pepsi} + \langle\textbf{w}_{Netflix,A} \cdot \textbf{w}_{Coca-cola,P}\rangle x_{Netflix}x_{Coca-cola} + \langle\textbf{w}_{Netflix,A} \cdot \textbf{w}_{Gatorade,P}\rangle x_{Netflix}x_{Gatorade} \\
&+ \langle\textbf{w}_{Netflix,G} \cdot \textbf{w}_{Male,P}\rangle x_{Netflix}x_{Male} + \langle\textbf{w}_{Netflix,G} \cdot \textbf{w}_{Female,P}\rangle x_{Netflix}x_{Female} + \langle\textbf{w}_{Spotify,A} \cdot \textbf{w}_{Pepsi,P}\rangle x_{Spotify}x_{Pepsi} \\
&+ \langle\textbf{w}_{Spotify,A} \cdot \textbf{w}_{Coca-cola,P}\rangle x_{Spotify}x_{Coca-cola} + \langle\textbf{w}_{Spotify,A} \cdot \textbf{w}_{Gatorade,P}\rangle x_{Spotify}x_{Gatorade} + \langle\textbf{w}_{Spotify,G} \cdot \textbf{w}_{Male,P}\rangle x_{Spotify}x_{Male} \\
&+ \langle\textbf{w}_{Spotify,G} \cdot \textbf{w}_{Female,P}\rangle x_{Spotify}x_{Female} + \langle\textbf{w}_{Facebook,A} \cdot \textbf{w}_{Pepsi,P}\rangle x_{Facebook}x_{Pepsi} + \langle\textbf{w}_{Facebook,A} \cdot \textbf{w}_{Coca-cola,P}\rangle x_{Facebook}x_{Coca-cola} \\
&+ \langle\textbf{w}_{Facebook,A} \cdot \textbf{w}_{Gatorade,P}\rangle x_{Facebook}x_{Gatorade} + \langle\textbf{w}_{Facebook,G} \cdot \textbf{w}_{Male,P}\rangle x_{Facebook}x_{Male} + \langle\textbf{w}_{Facebook,G} \cdot \textbf{w}_{Female,P}\rangle x_{Facebook}x_{Female} 
\end{aligned}
$$

For each impression, we are only concerned with the terms containing the features represented in the impression, since these $x$ values are equal to one, while all other terms become zero.
    
Our first impression (Netflix, Pepsi, Male) would therefore give us:
$$
\begin{aligned}
s &= \langle\textbf{w}_{Netflix,A} \cdot \textbf{w}_{Pepsi,P}\rangle + \langle\textbf{w}_{Netflix,G} \cdot \textbf{w}_{Male,P}\rangle + \langle\textbf{w}_{Pepsi,G} \cdot \textbf{w}_{Male,A}\rangle\\
&= (0.9*1 + -0.2*-0.3 + -0.2*-0.1) + (-0.6*-0.4 + 0*0.8 + -0.3*1.5) + (-0.4*-0.9 + -1*1.3 + 0*-1)  \\
&= 0.98 + -0.21 + -0.94 \\
&= -0.17
\end{aligned}
$$


Plugging this value into the logit transformation, we get a predicted CTR of:
$$\frac{1}{1+e^{-s}} = \frac{1}{1+e^{0.17}} \approx 0.46$$
    
Since this value is closer to 0 than 1, we would ultimately predict a non-click (incorrectly). However, this predicted value is vital for our log-loss calculation, which we will see momentarily.

Below we calculate the value of $s$ and predicted CTR for each impression:

In [27]:
W_p = sc.broadcast(W_P)
W_a = sc.broadcast(W_A)
W_g = sc.broadcast(W_G)

def predict(x):
    imp = x[0]
    res = x[2]
    pub = x[4]
    adv = x[6]
    gen = x[8]
    #define appropriate latent vectors to use in calculations
    if pub == '1':
        w_pa = W_p.value[0]
        w_pg = W_p.value[1]
    elif pub == '2':
        w_pa = W_p.value[2]
        w_pg = W_p.value[3]
    else:
        w_pa = W_p.value[4]
        w_pg = W_p.value[5]
    if adv == '1':
        w_ap = W_a.value[0]
        w_ag = W_a.value[1]
    elif adv == '2':
        w_ap = W_a.value[2]
        w_ag = W_a.value[3]
    else:
        w_ap = W_a.value[4]
        w_ag = W_a.value[5]
    if gen == '1':
        w_gp = W_g.value[0]
        w_ga = W_g.value[1]
    else:
        w_gp = W_g.value[2]
        w_ga = W_g.value[3]
    #calculate s using latent vectors
    s = (np.dot(w_pa, w_ap) + np.dot(w_pg, w_gp) + np.dot(w_ag, w_ga)).round(2)
    #calculate CTR
    CTR = np.divide(1,1+np.exp(-s)).round(2)
    yield imp,res, pub, adv, gen, s, CTR

In [28]:
resultRDD = toyRDD.flatMap(predict).cache()
resultRDD.collect()

[('1', '1', '1', '1', '1', -0.17, 0.46),
 ('2', '1', '2', '1', '1', 1.63, 0.84),
 ('3', '0', '3', '2', '2', -2.42, 0.08),
 ('4', '0', '2', '3', '1', 1.29, 0.78),
 ('5', '1', '3', '3', '2', -1.49, 0.18),
 ('6', '0', '3', '1', '2', -0.49, 0.38),
 ('7', '1', '1', '2', '2', -0.69, 0.33)]

Summarized in a table, we have:

<table>
<th>Impression</th>
<th>$s$</th>
<th>Predicted CTR</th>
<th>True Response</th>
<th>Correct?</th>
<tr><td>(Netflix, Pepsi, Male)</td><td>-0.17</td><td>$\approx 0.46$</td><td>1</td><td>No</td></tr>
<tr><td>(Spotify, Pepsi, Male)</td><td>1.63</td><td>$\approx 0.84$</td><td>1</td><td>Yes</td></tr>
<tr><td>(Facebook, Gatoriade, Female)</td><td>-2.42</td><td>$\approx 0.08$</td><td>0</td><td>Yes</td></tr>
<tr><td>(Spotify, Coca-cola, Male)</td><td>1.29</td><td>$\approx 0.78$</td><td>0</td><td>No</td></tr>
<tr><td>(Facebook, Coca-cola, Female)</td><td>-1.49</td><td>$\approx 0.18$</td><td>1</td><td>No</td></tr>
<tr><td>(Facebook, Pepsi, Female)</td><td>-0.49</td><td>$\approx 0.38$</td><td>0</td><td>Yes</td></tr>
<tr><td>(Netflix, Gatorade, Female)</td><td>-0.69</td><td>$\approx 0.33$</td><td>1</td><td>No</td></tr>
<table>

    
__4)__ We can now calculate the log-loss of the model. While this is not necessary in order to find the optimum latent vectors for our model, we show the calculation here using the equation:
$$ LogLoss = - \frac{1}{n} \sum_{i=1}^n [y_i \cdot log_e(\hat{y_i}) + (1-y_i) \cdot log_e(1-\hat{y_i})] $$

Simply put, if our True Response ($y_i$) for an impression is 0, we add the log-value of `1-Predicted CTR` to our sum; if our True Response ($y_i$) for an impression is 1, we add the log-value of `CTR`.    

Using our values from our table above, we have:

$$ \begin{aligned}
LogLoss =& - \frac{1}{7}[log_e(0.46) + log_e(0.84) + log_e(0.92) + log_e(0.22) + log_e(0.18) + log_e(0.62)+ log_e(0.33)]\\
=& 0.8356
\end{aligned}
$$

Our hand calculation is confirmed below:

In [29]:
def log_loss(imp):
    #helper function to calculate log(CTR) or log(1-CTR) depending on true response
    resp = imp[1]
    pred = imp[6]
    if resp == '0':
        yield 1, (np.log(1-pred), 1)
    else:
        yield 1, (np.log(pred), 1)

def sum_loss(x, y):
    #function to add log-losses, counts
    return (x[0]+y[0],x[1]+y[1])

loss = resultRDD.flatMap(log_loss).reduceByKey(lambda x, y: sum_loss(x, y))\
    .map(lambda x: -np.divide(x[1][0], x[1][1])).collect()
loss

[0.8356983388241626]

__5)__ We then would optimize our weights using gradient descent. In their implementation outlined in their paper, Juan, Zhuang, Chin, & Lin update the weight matrix of latent vectors using stochastic gradient descent. We will replicate this method with our toy sample here.

At each step, we choose a data point and update $\textbf{w}_{j_1,f_2}$ and $\textbf{w}_{j_2, f_1}$ for each non-zero pair of features $f_1 \epsilon j_1$ and $f_2 \epsilon j_2$. We first calculate the subgradients: 
$$\textbf{g}_{j_1,f_2} = \nabla \textbf{w}_{j_1,f_2} f(\textbf{w}) = \lambda \cdot \textbf{w}_{j_1,f_2} + \kappa \cdot \textbf{w}_{j_2,f_1}x_{j_1}x_{j_2} $$
$$\textbf{g}_{j_2,f_1} = \nabla \textbf{w}_{j_2,f_1} f(\textbf{w}) = \lambda \cdot \textbf{w}_{j_2,f_1} + \kappa \cdot \textbf{w}_{j_1,f_2}x_{j_1}x_{j_2} $$
where $$\kappa = \frac{-\bar{y}}{1+exp(\bar{y}\phi(\textbf{wx}))}$$

with $\bar{y}=1$ for a positive click response and $\bar{y}=-1$ for a negative click response, as before.

For simplicity here, we ignore the regularization term and consider the first datapoint- the impression of (Netflix, Pepsi, Male). We calculate $\kappa = -0.54$, and calculate the first pair of subgradients:
    $$\textbf{g}_{Pepsi, P} = \kappa\textbf{w}_{Netflix, A}$$ 
    $$\textbf{g}_{Netflix, A} = \kappa\textbf{w}_{Pepsi, P}$$
    
(Note: below we only continue to demonstrate how to update the latent vector $\textbf{w}_{Pepsi,P}$. Updating all other latent vectors is analagous.)
    
Plugging in our known and calculated values, we have:
    
$$\textbf{g}_{Pepsi, P}= -0.54 \times \begin{bmatrix}0.9\\-0.2\\-0.2 \end{bmatrix} = \begin{bmatrix}-0.488\\0.108\\0.108 \end{bmatrix} $$

Next, for each coordinate $d = {1,...,k}$, the sum of squared gradient is accumulated:
$$ (G_{j_1,f_2})_d \leftarrow (G_{j_1,f_2})_d + (\textbf{g}_{j_1,f_2})_d^2$$ where $(G_{j_1,f_2})_d$ are initialized as all 1s. Thus, we have:
$$ (G_{Pepsi,P}) = \begin{bmatrix}1\\1\\1\end{bmatrix} + \begin{bmatrix}(-0.488)^2\\(0.108)^2\\(0.108)^2\end{bmatrix} = \begin{bmatrix}1.2383\\1.0117\\1.0117\end{bmatrix}  $$
    
Finally, we update our latent vector $\textbf{w}_{j_1,f_2}$ by:
    $$(\textbf{w}_{j_1,f_2})_d \leftarrow (\textbf{w}_{j_1,f_2})_d - \frac{\eta}{\sqrt{(G_{j_1,f_2})_d}} \textbf{g}_{j_1,f_2} $$
where $\eta$ is a specified learning rate. For this example, we use $\eta = 0.1$, and get the following update to our latent vector, $\textbf{w}_{Pepsi,P}$:

$$\textbf{w}_{Pepsi,P} = \begin{bmatrix}1\\-0.3\\-0.1 \end{bmatrix} - \begin{bmatrix}\frac{0.1}{\sqrt{1.2383}} \cdot -0.488\\\frac{0.1}{\sqrt{1.0117}} \cdot 0.108\\\frac{0.1}{\sqrt{1.0117}} \cdot 0.108 \end{bmatrix} = \begin{bmatrix}1.04\\-0.31\\-0.11 \end{bmatrix}$$

Using the same impression, we similarly update the latent vectors $\textbf{w}_{Netflix,A}$, $\textbf{w}_{Netflix,G}$, $\textbf{w}_{Male,P}$, $\textbf{w}_{Male,A}$, and $\textbf{w}_{Pepsi,G}$.
These newly updated latent vectors are calculated below:

In [30]:
def kappa(x):
    #function to calculate kappa value based on true response and calculated s value
    if x[1]=='1':
        y=1
    else:
        y=-1
    return -np.divide(y,1 + np.exp(y*x[5])) #kappa calculation

def gradient(x):
    #function to calculate gradient update for latent vectors of an impression
    #need response variables to be +1 or -1
    k = kappa(x)
    pub = x[2]
    adv = x[3]
    gen = x[4]
    #extract appropriate latent vectors to update, given the impression
    if pub == '1':
        w_pa = W_p.value[0]
        w_pg = W_p.value[1]
    elif pub == '2':
        w_pa = W_p.value[2]
        w_pg = W_p.value[3]
    else:
        w_pa = W_p.value[4]
        w_pg = W_p.value[5]
    if adv == '1':
        w_ap = W_a.value[0]
        w_ag = W_a.value[1]
    elif adv == '2':
        w_ap = W_a.value[2]
        w_ag = W_a.value[3]
    else:
        w_ap = W_a.value[4]
        w_ag = W_a.value[5]
    if gen == '1':
        w_gp = W_g.value[0]
        w_ga = W_g.value[1]
    else:
        w_gp = W_g.value[2]
        w_ga = W_g.value[3]
    #calculate subgradients
    g_pa = k*w_ap
    g_pg = k*w_gp
    g_ap = k*w_pa
    g_ag = k*w_ga
    g_gp = k*w_pg
    g_ga = k*w_ag
    #accumulate sum of squared gradients, add to G vector (initialized as ones)
    G_pa = np.ones(3) + g_pa**2
    G_pg = np.ones(3) + g_pg**2
    G_ap = np.ones(3) + g_ap**2
    G_ag = np.ones(3) + g_ag**2
    G_gp = np.ones(3) + g_gp**2
    G_ga = np.ones(3) + g_ga**2
    
    #update latent vectors
    w_pa = w_pa - np.divide(0.1,np.sqrt(G_pa))*g_pa
    w_pg = w_pg - np.divide(0.1,np.sqrt(G_pg))*g_pg
    w_ap = w_ap - np.divide(0.1,np.sqrt(G_ap))*g_ap
    w_ag = w_ag - np.divide(0.1,np.sqrt(G_ag))*g_ag
    w_gp = w_gp - np.divide(0.1,np.sqrt(G_gp))*g_gp
    w_ga = w_ga - np.divide(0.1,np.sqrt(G_ga))*g_ga
    #yield features (to know which vectors to update in weight matrix) and updated latent vectors
    yield  pub, adv, gen, w_pa, w_pg, w_ap, w_ag, w_gp, w_ga
    
# demonstrate, specifying first impression to check hand calculations
update = sc.parallelize(resultRDD.takeOrdered(1)).flatMap(gradient).collect()
update

[('1',
  '1',
  '1',
  array([ 0.94767801, -0.2160607 , -0.20541602]),
  array([-0.62120264,  0.03980592, -0.2368895 ]),
  array([ 1.04386801, -0.31078469, -0.11078469]),
  array([-0.44386801, -0.94237342, -0.04767801]),
  array([-0.43094634,  0.8       ,  1.4839393 ]),
  array([-0.92120264,  1.25232199, -1.        ]))]

We note that the third vector here, representing the updated `PepsiP` latent vector,  matches our hand calculation (which rounded to the second decimal place). We must also pass along the features in order to ensure we update the correct latent vectors in the weight matrix. In the case of the first impression, we emit '1', '1', '1' for the features, so we know to update NetflixA (`W_P[0]`), Netflix G (`W_P[1]`), PepsiP(`W_A[0]`), PepsiG (`W_A[1]`), MaleP(`W_G[0]`), and MaleA (`W_G[1]`) in the weight matrix:  

In [31]:
def updateWeights(W_P, W_A, W_G, update):
# Function that takes current weight matrices, and updated vectors emitted after SGD
# and updates Weight Matrices
    #take first feature (Publisher), and update W_P latent vectors accordingly
    if update[0][0] == '1':
        W_P[0] = update[0][3]
        W_P[1] = update[0][4]
    elif update[0][0] == '2':
        W_P[2] = update[0][3]
        W_P[3] = update[0][4]
    else:
        W_P[4] = update[0][3]
        W_P[5] = update[0][4]
    #take second feature (Advertiser), and update W_A latent vectors accordingly
    if update[0][1] == '1':
        W_A[0] = update[0][5]
        W_A[1] = update[0][6]
    elif update[0][1] == '2':
        W_A[2] = update[0][5]
        W_A[3] = update[0][6]
    else:
        W_A[4] = update[0][5]
        W_A[5] = update[0][6]
    #take third feature (Gender), and update W_G latent vectors accordingly
    if update[0][2] == '1':
        W_G[0] = update[0][7]
        W_G[1] = update[0][8]
    else:
        W_G[2] = update[0][7]
        W_G[3] = update[0][8]
    return W_P, W_A, W_G

Below we print the updated matrices of latent vectors that would be broadcast back out for the next iteration:

In [32]:
updateWeights(W_P, W_A, W_G, update)

(array([[ 0.94767801, -0.2160607 , -0.20541602],
        [-0.62120264,  0.03980592, -0.2368895 ],
        [-0.1       , -0.5       , -1.2       ],
        [-1.1       ,  0.2       ,  1.2       ],
        [ 1.4       , -0.7       ,  0.7       ],
        [ 0.        ,  0.8       ,  0.1       ]]),
 array([[ 1.04386801, -0.31078469, -0.11078469],
        [-0.44386801, -0.94237342, -0.04767801],
        [-0.2       , -1.        , -1.7       ],
        [ 0.7       , -0.3       , -1.1       ],
        [ 0.1       ,  1.3       , -0.2       ],
        [ 0.5       ,  0.2       ,  0.5       ]]),
 array([[-0.43094634,  0.8       ,  1.4839393 ],
        [-0.92120264,  1.25232199, -1.        ],
        [ 0.8       , -1.5       ,  0.3       ],
        [ 0.4       ,  0.7       ,  0.5       ]]))

__6)__ We repeat 3-5 for a specified number of iterations or until some convergence criteria is met for the latent vectors. In each iteration we would use a different impression to update the latent vectors. Therefore, our code from above must be slightly modified (to choose an impression at random, and not explicitly the first impression. Below is the code we would run for 10 iterations:

In [35]:
W_P = np.random.uniform(-1.7,1.7, size=(6,3)).round(1)
W_A = np.random.uniform(-1.7,1.7, size=(6,3)).round(1)
W_G = np.random.uniform(-1.7,1.7, size=(4,3)).round(1)
for i in range(10):
    print('Iteration:', i+1)
    W_p = sc.broadcast(W_P)
    W_a = sc.broadcast(W_A)
    W_g = sc.broadcast(W_G)
    resultRDD = toyRDD.flatMap(predict).cache()
    loss = resultRDD.flatMap(log_loss).reduceByKey(lambda x, y: sum_loss(x, y))\
        .map(lambda x: -np.divide(x[1][0], x[1][1])).collect()
    print('Loss:', loss[0])
    update = sc.parallelize(resultRDD.takeSample(True,1)).flatMap(gradient).collect()
    updateWeights(W_P, W_A, W_G, update)
    
print('Final Weight Matrices:')
print('W_P:\n', W_P,'\nW_A:\n', W_A,'\nW_G:\n', W_G)

Iteration: 1
Loss: 0.6674505942985505
Iteration: 2
Loss: 0.6494112620568574
Iteration: 3
Loss: 0.655285193107078
Iteration: 4
Loss: 0.6289630447180298
Iteration: 5
Loss: 0.6460073554561067
Iteration: 6
Loss: 0.6310762578752105
Iteration: 7
Loss: 0.6310762578752105
Iteration: 8
Loss: 0.6269998104577843
Iteration: 9
Loss: 0.6314429931312981
Iteration: 10
Loss: 0.6278931538743446
Final Weight Matrices:
W_P:
 [[ 0.43168523  0.74967893  0.34789988]
 [ 0.7024423   0.99953272  0.52493794]
 [-0.9         1.4        -1.5       ]
 [-1.6        -0.4        -1.4       ]
 [-0.04794223  1.54197082 -1.46481178]
 [-0.4055017   0.06581357  1.12199341]] 
W_A:
 [[ 0.40556162 -0.44751152  0.65177044]
 [ 0.89036241 -1.50447862  0.95611675]
 [ 1.45286369 -0.16237191  0.25864327]
 [ 1.27069659 -0.86144194  1.32753492]
 [-0.3         0.4         1.6       ]
 [-0.6        -0.7        -0.7       ]] 
W_G:
 [[-1.29278909 -0.38949718  1.60583298]
 [ 0.30907984 -1.41524401  1.00969589]
 [ 0.17304778 -0.42320007 -1.

We note that stochastic gradient descent does not always improve the loss from iteration to iteration, as only a single impression is used to update after each iteration. For this and other reasons, we consider using other methods of gradient descent moving forward.

#### Sources:
- https://www.csie.ntu.edu.tw/~r01922136/slides/ffm.pdf
- https://www.csie.ntu.edu.tw/~cjlin/papers/ffm.pdf
- https://www.youtube.com/watch?v=1cRGpDXTJC8

# __Section 3__ - EDA & Challenges

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts